In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/Users/tanishqsharma/PycharmProjects/PythonProject1/data/criteo-uplift-v2.1.csv')

In [3]:
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
0,12.616365,10.059654,8.976429,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
1,12.616365,10.059654,9.002689,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
2,12.616365,10.059654,8.964775,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
3,12.616365,10.059654,9.002801,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
4,12.616365,10.059654,9.037999,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0


In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier


# Define columns
binary_cols = ['treatment', 'conversion', 'visit', 'exposure']
feature_cols = [col for col in df.columns if col.startswith('f')] + ['visit', 'exposure']
continuous_cols = [col for col in feature_cols if col not in binary_cols]

# Apply standard scaling to continuous features
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[continuous_cols] = scaler.fit_transform(df[continuous_cols])

# Prepare inputs
X = df_scaled[feature_cols]
y = df_scaled['conversion']
treatment = df_scaled['treatment']
# Split data
X_treated = X[treatment == 1]
y_treated = y[treatment == 1]

X_control = X[treatment == 0]
y_control = y[treatment == 0]

# Train models
model_treated = XGBClassifier(eval_metric="logloss", use_label_encoder=False)
model_treated.fit(X_treated, y_treated)

model_control = XGBClassifier(eval_metric="logloss", use_label_encoder=False)
model_control.fit(X_control, y_control)



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [14:16:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [14:17:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [16]:
import shap
import numpy as np

# Sample input from dataset (can replace this with new user input later)
sample_input = X.sample(1, random_state=42)

# SHAP for treated model
explainer_treated = shap.Explainer(model_treated)
shap_treated = explainer_treated(sample_input)

# SHAP for control model
explainer_control = shap.Explainer(model_control)
shap_control = explainer_control(sample_input)

# Compute per-feature uplift contributions
uplift_contribs = shap_treated.values - shap_control.values

# Package into DataFrame
contrib_df = pd.DataFrame({
    'feature': sample_input.columns,
    'uplift_contribution': uplift_contribs[0]
}).sort_values(by='uplift_contribution', ascending=False)

# Print results
print("\n🔍 Per-Feature Contribution to Uplift for the Sample Input:\n")
print(contrib_df)




🔍 Per-Feature Contribution to Uplift for the Sample Input:

     feature  uplift_contribution
10       f10            73.562363
9         f9             6.550283
3         f3             2.991191
0         f0             0.595290
2         f2             0.550672
5         f5             0.049509
7         f7             0.011395
1         f1             0.002116
11       f11            -2.193917
6         f6            -3.157839
13  exposure            -3.849447
4         f4           -12.886565
8         f8           -21.720749
12     visit          -106.897644


In [17]:
import joblib

# Save treated model
joblib.dump(model_treated, 'model_treated.pkl')

# Save control model
joblib.dump(model_control, 'model_control.pkl')

# Save scaler
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']